# Lesson 3: Vectorstores and embeddings

# Vectorstore ingestion

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings();

await embeddings.embedQuery("This is some sample text");

[
   -0.010372737,  0.0023619137,  -0.000738098,  -0.010897608,   -0.011474964,
    0.022949928,  -0.014656986,  0.0017665146,   -0.01755689,    -0.01927584,
    0.005156845,    0.03419526,  -0.012183538,  0.0019223352,    0.004691023,
    0.013141425,   0.024642633,  0.0017369906,   0.004533562,   -0.006232828,
   -0.005114199, -0.0006815105,  -0.008187967,    0.01393529,   -0.008916224,
  -0.0040480574, -0.0007487594,  -0.019617004,  0.0040316554,  -0.0017140276,
    0.016061014,  -0.021873945, -0.0007725426,  -0.022254474,    0.006242669,
    0.007033254,  -0.011205968,  -0.013416981,   0.028815346,   -0.023619136,
    0.006875793,   0.017517526,   0.017307578, -0.0059310277,  -0.0038676336,
   -0.014368308, -0.0017369906, -0.0059769535,   -0.00790585,    0.010425224,
     0.03275187, -0.0008668551,   -0.02402591, -0.0033854095, -0.00083446084,
    0.002598105,  -0.022451302,   0.025141258,   0.019761344,    0.010805755,
   -0.011711155, -0.0025505386,  0.0024373636, -0.0073088105, 

In [3]:
import { similarity } from "ml-distance";

const vector1 = await embeddings.embedQuery(
    "What are vectors useful for in machine learning?"
);
const unrelatedVector = await embeddings.embedQuery(
    "A group of parrots is called a pandemonium."
);

In [4]:
similarity.cosine(vector1, unrelatedVector);

0.6957264527346025

In [5]:
const similarVector = await embeddings.embedQuery(
    "Vectors are representations of information."
);

similarity.cosine(vector1, similarVector);

0.8588144744020122

In [6]:
// Peer dependency
import * as parse from "pdf-parse";
import { PDFLoader } from "langchain/document_loaders/fs/pdf";
import { 
    RecursiveCharacterTextSplitter
} from "langchain/text_splitter";

const loader = new PDFLoader("./data/MachineLearning-Lecture01.pdf");

const rawCS229Docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 128,
  chunkOverlap: 0,
});

const splitDocs = await splitter.splitDocuments(rawCS229Docs);

In [7]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);

In [8]:
await vectorstore.addDocuments(splitDocs);

In [9]:
const retrievedDocs = await vectorstore.similaritySearch(
    "What is deep learning?", 
    4
);

const pageContents = retrievedDocs.map(doc => doc.pageContent);

pageContents

[
  "piece of research in machine learning, okay?",
  "are using a learning algorithm, perhaps without even being aware of it.",
  "some of my own excitement about machine learning to you.",
  "of the class, and then we'll start to talk a bit about machine learning."
]

# Retrievers

In [10]:
const retriever = vectorstore.asRetriever();

In [11]:
await retriever.invoke("What is deep learning?")

[
  Document {
    pageContent: "piece of research in machine learning, okay?",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",
          IsAcroFormPresent: false,
          IsXFAPresent: false,
          Title: "",
          Author: "",
          Creator: "PScript5.dll Version 5.2.2",
          Producer: "Acrobat Distiller 8.1.0 (Windows)",
          CreationDate: "D:20080711112523-07'00'",
          ModDate: "D:20080711112523-07'00'"
        },
        metadata: Metadata { _metadata: [Object: null prototype] },
        totalPages: 22
      },
      loc: { pageNumber: 8, lines: { from: 2, to: 2 } }
    }
  },
  Document {
    pageContent: "are using a learning algorithm, perhaps without even being aware of it.",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",